# 1. Colab Setting

In [3]:
# Set the directory path where training data and fine-tuned model files exist
path = "/content/drive/MyDrive/Solution Challenge/Algorithm/"

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 19.2 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.1
    Uninstalling graphviz-0.20.1:
      Successfully uninstalled graphviz-0.20.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 344.5/344.5 KB 28.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp39-cp39-linux_x86_64.whl size=680532 sha256=6f4cc2d32c376b115b97672242408a5ce041ae03048c7ad30ba096afba9092d6
  Stored in directory: /root/.cache/pip/wheels/47/17/70/b257bc53879a458c4bfcc900e89271aa8b4f19366a54bd2455
Successfully built gluonnlp
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [5]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-5do6ep9v
  Running command git clone --filter=blob:none --quiet 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-5do6ep9v
  Resolved https://****@github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 KB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.7/54.7 MB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 87.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 48.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 881.9/881.9 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 84.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [6]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [7]:
# KoBERT
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

# Transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [8]:
# Using GPU
device = torch.device("cuda:0")

In [9]:

# BERT model, load vocabulary
bertmodel, vocab = get_pytorch_kobert_model()

/content/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


# 2. Dataset Pre-processing

In [11]:
import pandas as pd
chatbot_data = pd.read_excel(path+'/conversation_data_set.xlsx')

In [12]:
data_list = []
for q, label in zip(chatbot_data.iloc[:,1], chatbot_data.iloc[:,2])  :
    data = []
    data.append(q)
    data.append(str(int(label)))

    data_list.append(data)

In [13]:
# Check few sentences for train and test
print(data_list[0])
print(data_list[400])
print(data_list[800])
print(data_list[1200])
print(data_list[1600])
print(data_list[2000])
print(data_list[-1])

['여보세요 응 이런 사람은 알고 계세요. 건물이나 돈을 범죄 사겠다는 권고하는 과정에서 대량의 신용카드를 대포통령이 발견이 됐는데 그중 문제가 되는게 거기서 지금 명의로 된 하나은행 통장과 농협 통장에 발견이 됐어요. 그 두 개의 통장 개설에 대해서 아시는 내용 있으세요 아니요? 전혀 없어요음 그 두 개의 통장을 작년 3월 3일 경기도 광명시 철산동에서 바둑 계절을 개설된 계좌인데 어 전혀 모르셨어요음네 알겠습니다. 끝없이 통화 불편하시면 말씀을 해 주세요음 제출 계좌를 불법 현장에서 알 수 있기 때문에 동결 처리와 동시에 북구로 환수 조치했고요. 금융 운동은 제주공 결과 또 계좌번호도 확실히 명의 계정이한테 저희가 확인을 했어요. 제가 어 저에게 연락드린 이유는 어? 본인이 직접 통장을 개설하셔서 일당들에게 금전적인 대가를 받고 통장을 판매하신 건 아닌지 일단 아니면은 제 개인 정보가 유출이 되면서 명의 조정 당하실 피해자 분이신지에 대해서 저는 연락을 드린 거예요. 통장을 판매는 양봉하신 적 있으세요 아니요? 통장을 판매를 양도 하시는 것도 아니라고 보이는 하셨고도 모른다고 보기는 하셨는데 그러면은 사이에 지갑이나 핸드폰이랑 개인정보가 있을 만한 물건을 분실하신 데가 있을까요 아니요 없어요음 질문을 좀 똑바로 좀 해주세요 장난치지 마시고. 장난 아닌데요. 네, 알겠습니다. 자 그러면은 아 혹시 이런 쇼핑몰 사이트를 이용하세요 11번가나 옥션 지마켓. 라면 그냥 저희가 이걸 왜 못 잡았냐면 대부분 평소들이 이런 식품으로 사이트에서 개인 정보였지 상당히 많이 됐더라고요. 안녕! 네 저 지금 이거 조사를 어 한 두 시간 정도 할 건데 여기 괜찮으세요? 자.', '1']
['서울중앙지방검찰청에서 전화드린 거예요네네 자 현재 저희 검찰에서 김용식 주범으로 인한 금융범죄 사기단의 검거한 상태예요. 자이 검거 현장에서 대량의 신용카드가 대포통장을 압수하던 과정에 연기로 돼 이제 농협 하나는 통장 두 점이 발견되어 조석제 연락드린 거고요. 자 문의 혹시이 통장 개별 거

# 3. Split Data into Train and Test

In [14]:
# Split data into train and test
from sklearn.model_selection import train_test_split
                                                         
dataset_train, dataset_test = train_test_split(data_list, test_size=0.25, random_state=0)

In [15]:
# Check the number of sentences
print(len(dataset_train))
print(len(dataset_test))

1645
549


# 4. Prepare KoBERT and Input Data

In [16]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [17]:
# Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 10
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [18]:
# Tokenization
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [19]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.9/dist-packages/torch/utils/data/dataloader.py:478: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


# 5. Prepare KoBERT classifier

In [20]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=2,   # Modify the number of classes
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [21]:
# Load KoBERT model
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

# Set optimizer and schedule
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

# Define a function for accuracy measure
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

# 6. Test New Sentences

In [22]:
import numpy as np

In [23]:
# Load trained pt file
model_conv = torch.load("/content/drive/MyDrive/KAIST/2022winter/voice_phishing/conversation_model_res.pt")

In [24]:
# Tokenization
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)
def softmax(a):
    exp_a = np.exp(a)
    sum_exp_a = np.sum(exp_a)
    y = exp_a / sum_exp_a

    return y


def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model_conv.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model_conv(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()
            print(logits)
            if np.argmax(logits) == 0:
                test_eval.append("일반 대화")
            elif np.argmax(logits) == 1:
                test_eval.append("보이스피싱")
            result = softmax(logits)
            
    result_list = result.tolist()

    return result_list

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


# 7. Prepare Server with Flask

In [25]:
!pip install flask

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [31]:
!pip install pyngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 761.3/761.3 KB 45.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-5.2.1-py3-none-any.whl size=19790 sha256=b9325370b6599e58b2b0e7638d930d70a0f9112e54d920c8a05a6ca696f1d78f
  Stored in directory: /root/.cache/pip/wheels/f6/89/59/49d4249e00957e94813ac136a335d10ed2e09a856c5096f95c
Successfully built pyngrok


In [32]:
from flask import Flask, jsonify, request, Response

In [33]:
app = Flask(__name__)

@app.route('/classify/')
def classify():
    # Handle GET request here
    # sentence = request.get_json()
    print("request recieved!")
    try:
      # print("handle...")
      data = {'message': 'Hello, world!'}
      # print(request.get_json())
      sentence = request.get_json()
      # print("data achieved!")
      # print(sentence)
      # print(sentence["data"])
      data = predict(sentence["data"])
      print("predict success!")
      # print(sentence)
    except Exception as e:
      print(f" Err: {e}")
      data = {'message': 'Hello, world!'}

    
    # data = {'message': 'Hello, world!'}
    return jsonify(data[1])

In [34]:
# Start ngrok tunnel
from pyngrok import ngrok
public_url = ngrok.connect(port=80).public_url
print("Public URL:", public_url)

Public URL: http://b17d-34-143-209-155.ngrok.io


In [35]:
app.run(port=80)

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:80
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [29/Mar/2023 17:56:11] "GET / HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [29/Mar/2023 17:56:11] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [29/Mar/2023 17:56:13] "GET / HTTP/1.1" 404 -


In [36]:
# Import necessary libraries
import socket
import requests

# Get the public IP address of the Colab runtime
s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
s.connect(("8.8.8.8", 80))
public_ip_address = s.getsockname()[0]
s.close()

# Print the IP address
print("Public IP address of the Colab runtime:", public_ip_address)

Public IP address of the Colab runtime: 172.28.0.12


In [38]:
!ngrok authtoken 2Bd65SUazYWfMaAm9a2LhqSRps8_4Seeutz6NQiD7v1S3PU2T

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml
